<a href="https://colab.research.google.com/github/yatindma/Automated-Response-Suggestion-for-Email/blob/master/Smart_Gmail_Reply.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h2> Overview </h2>


Here we'll try to predict the Short email responses.<br>
As most of the emails have shorter replies.<br>
So that it can be easy to reply, without the hassle of typing.<br>
It can help in saving the time of the person.<br>

########################################################

Will update later.
########################################################







In [0]:
import pandas as pd
import numpy as np

In [0]:
data = pd.read_csv('qa_dataset.csv', encoding = "ISO-8859-1", low_memory=False) 

In [25]:
data.head()

,ArticleTitle,Question,Answer,DifficultyFromQuestioner,DifficultyFromAnswerer,ArticleFile
0,Alessandro_Volta,Was Volta an Italian physicist?,yes,easy,easy,data/set4/a10
1,Alessandro_Volta,Is Volta buried in the city of Pittsburgh?,no,easy,easy,data/set4/a10
2,Alessandro_Volta,Did Volta have a passion for the study of elec...,yes,easy,medium,data/set4/a10
3,Alessandro_Volta,What is the battery made by Volta credited to be?,the first cell,medium,medium,data/set4/a10
4,Alessandro_Volta,What important electrical unit was named in ho...,the volt,medium,medium,data/set4/a10


<b>
Article title can be used while giving the sentence for tokenizing later 
Question do we need to preprocess the data
Answer mai kitne repeated h
</b>


<b>Get the data to Visulize and preprocess it.<b>

In [0]:


# data = pd.read_csv('qa_dataset.csv')

In [17]:
from io import StringIO
s=str(bytes_data,'utf-8')

data = StringIO(s) 

df=pd.read_csv(data)

UnicodeDecodeError: ignored

In [18]:
df.head()

""
